In [1]:
#from cityscapes_dataset import CityscapesDataset
from nyuv2_dataset import NYUv2Dataset
from cityscapes_dataset import CityscapesDataset
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from densenet import DenseNet

from trainer import Trainer
from utils import visualize_results_multitask, count_params, AngleDistance, MeanAbsoluteRelativeError

2025-03-01 18:32:20.408484: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 18:32:20.473597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-01 18:32:21.578888: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
LABELS = 7
BATCH_SIZE = 8

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [27]:
print("NYUv2 Dataset")
nyuv2 = NYUv2Dataset(root="../dataset/nyuv2_preprocessed")
train_dl = DataLoader(nyuv2, batch_size=1, shuffle=True)
for image, out in train_dl:
    print('Image: ' + str(list(image.shape)) + ',' + ' Label: ' + str(list(out['segmentation'].shape)) + ',' + ' Depth: ' + str(list(out['depth'].shape)) + ',' + ' Normals: ' + str(list(out['normal'].shape)))
    print(f'Image: {image.max().item()}, {image.min().item()}') 
    print('Label: ' +  str(out['segmentation'].max().item()) + ',' + str(out['segmentation'].min().item()))
    print('Depth: ' + str(out['depth'].max().item()) + ', ' + str(out['depth'].min().item()))
    print('Normals: ' + str(out['normal'].max().item()) + ', ' + str(out['normal'].min().item()))
    break
# ax, fig = plt.subplots(4, figsize=(10, 10))
# fig[0].imshow(image[0].permute(1, 2, 0))
# fig[1].imshow(label[0])
# fig[2].imshow(depth[0])
# fig[3].imshow(normals[0].permute(1, 2, 0))
# plt.show()

NYUv2 Dataset
Image: [1, 3, 288, 384], Label: [1, 288, 384], Depth: [1, 288, 384], Normals: [1, 3, 288, 384]
Image: 1.0, 0.0
Label: 7.0,-1.0
Depth: 9.968480110168457, 0.0
Normals: 0.9999846216542477, -0.9999846216542477


In [28]:
print("Cityscapes Dataset")
cityscapes = CityscapesDataset(root="../dataset/cityscapes_preprocessed")
train_dl = DataLoader(cityscapes, batch_size=1, shuffle=True)
for image, out in train_dl:
    print('Image: ' + str(list(image.shape)) + ',' + ' Label: ' + str(list(out['segmentation'].shape)) + ',' + ' Depth: ' + str(list(out['depth'].shape)))
    print(f'Image: {image.max().item()}, {image.min().item()}') 
    print('Label: ' +  str(out['segmentation'].max().item()) + ', ' + str(out['segmentation'].min().item()))
    print('Depth: ' + str(out['depth'].max().item()) + ', ' + str(out['depth'].min().item()))
    break

Cityscapes Dataset
Image: [1, 3, 128, 256], Label: [1, 128, 256], Depth: [1, 128, 256]
Image: 0.9725490196078431, 0.06274509803921569
Label: 6.0, -1.0
Depth: 0.4922102689743042, 0.0


In [ ]:
cityscapes_train = CityscapesDataset(root="../cityscapes_preprocessed", split="train",labels=LABELS)
train_dl = DataLoader(cityscapes_train, batch_size=BATCH_SIZE, shuffle=False)
for image, label, depth in train_dl:
    print(f'Image: {image.shape}, Label: {label.shape}, Depth: {depth.shape}')
    print(f'Image: {image.max()}, {image.min()}') 
    print(f'Label: {label.max()}, {label.min()}')
    print(f'Depth: {depth.max()}, {depth.min()}')
    break
ax, fig = plt.subplots(3, figsize=(10, 10))
fig[0].imshow(image[0].permute(1, 2, 0))
fig[1].imshow(label[0])
fig[2].imshow(depth[0])
plt.show()

cityscapes_val = CityscapesDataset(root="../cityscapes_preprocessed", split="val", labels=LABELS)
val_dl = DataLoader(cityscapes_val, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model = DenseNet(classes=LABELS)
print(f"{model.name} has {count_params(model)} parameters")
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
trainer = Trainer(model, opt)
trainer.train(train_dl, val_dl, epochs=2, save=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tests = 10
path = 'models/mtan/mtan_train10.pth'
model = MTAN(classes=LABELS)
model.load_state_dict(torch.load(path, weights_only=True))
model.to(device)
for i, (img, img_seg, img_depth) in enumerate(val_dl):
    visualize_results_multitask(model, img[0], img_seg[0], img_depth[0], device)
    if i == tests:
        break